In [10]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, RDF, RDFS, OWL #most common namespaces
import urllib.parse #for parsing strings to URI's

In [11]:
df=pd.read_csv('character_stats_id.csv')
df.head()


,Name,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total,Character ID
0,3-D Man,good,50,31,43,32,25,52,233,3_d_man
1,A-Bomb,good,38,100,17,80,17,64,316,a_bomb
2,Abe Sapien,good,88,14,35,42,35,85,299,abe_sapien
3,Abin Sur,good,50,90,53,64,84,65,406,abin_sur
4,Abomination,bad,63,80,53,90,55,95,436,abomination


In [12]:
#Check for missing values
df.isnull().sum()  

Name            0
Alignment       3
Intelligence    0
Strength        0
Speed           0
Durability      0
Power           0
Combat          0
Total           0
Character ID    0
dtype: int64

In [13]:
#define graph 
g = Graph()
#define namespaces
dbo = Namespace('http://dbpedia.org/ontology/')
schema = Namespace('http://schema.org/')
wd = Namespace('https://www.wikidata.org/wiki/')
ken = Namespace('https://w3id.org/um/ken4256/')
ken_character = Namespace('https://w3id.org/um/ken4256/character/')
ken_char_alignment =  Namespace('https://w3id.org/um/ken4256/alignment/')
cbo = Namespace('http://comicmeta.org/cbo/')

In [14]:
#Add triples to the graph

g = Graph()
for index, row in df.iterrows():
    g.add((URIRef(ken_character+row['Character ID']), RDF.type, URIRef(cbo.Character) )) 
    g.add((URIRef(ken_character+row['Character ID']), FOAF.name, Literal(row['Name'], lang = 'en') ))
    if (row['Alignment'] == row['Alignment']):
        g.add((URIRef(ken_character+row['Character ID']), ken.alignment, URIRef(ken_char_alignment+urllib.parse.quote(row['Alignment'])) ))
    g.add((URIRef(ken_character+row['Character ID']), wd.Q83500, Literal(int(row['Intelligence']), datatype=XSD['int']) ))
    g.add((URIRef(ken_character+row['Character ID']), wd.Q4386945, Literal(int(row['Strength']), datatype=XSD['int']) )) 
    g.add((URIRef(ken_character+row['Character ID']), wd.Q3711325, Literal(int(row['Speed']), datatype=XSD['int']) ))
    g.add((URIRef(ken_character+row['Character ID']), wd.Q47089006, Literal(int(row['Durability']), datatype=XSD['int']) )) 
    g.add((URIRef(ken_character+row['Character ID']), ken.power, Literal(int(row['Power']), datatype=XSD['int']) ))
    g.add((URIRef(ken_character+row['Character ID']), wd.Q650711, Literal(int(row['Combat']), datatype=XSD['int']) ))
    g.add((URIRef(ken_character+row['Character ID']), ken.total, Literal(int(row['Total']), datatype=XSD['int']) ))


In [15]:
#Define my own properties (with domain range, comment)
g.add((URIRef(ken.alignment), RDF.type, URIRef(OWL.DatatypeProperty)))
g.add((URIRef(ken.power), RDF.type, URIRef(OWL.DatatypeProperty)))
g.add((URIRef(ken.total), RDF.type, URIRef(OWL.DatatypeProperty)))
g.add((URIRef(ken.alignment), RDFS.comment, Literal("Defines the superhero's alignment, i.e. if they are good or bad characters.", lang="en")))
g.add((URIRef(ken.power), RDFS.comment, Literal("Defines the magnitude of the superhero's power.", lang="en")))
g.add((URIRef(ken.total), RDFS.comment, Literal("Defines how many points each superhero scores in total based on the aforementioned criteria.", lang="en")))
g.add((URIRef(ken.alignment), RDFS.domain, URIRef(cbo.Character)))
g.add((URIRef(ken.power), RDFS.domain, URIRef(cbo.Character)))
g.add((URIRef(ken.total), RDFS.domain, URIRef(cbo.Character)))
g.add((URIRef(ken.alignment), RDFS.range, XSD['string']))
g.add((URIRef(ken.power), RDFS.range, XSD['int']))
g.add((URIRef(ken.total), RDFS.range, XSD['int']))


In [16]:
#Bind the namespaces for prefixes for a more readable output
g.bind("foaf", FOAF)
g.bind("wd",wd)
g.bind("schema",schema)
g.bind("dbo",dbo)
g.bind("ken",ken)
g.bind("ken_character",ken_character)
g.bind("ken_char_alignment",ken_char_alignment)
g.bind("cbo",cbo)

In [17]:
#print(g.serialize(format='turtle').decode('UTF-8'))

In [18]:
#Save output
g.serialize('output_characters.ttl',format='turtle')